In [1]:
import numpy as np
import pandas as pd
import csv
from scamp import *
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense,Flatten,Softmax,Input
import keras.backend as K 

Using TensorFlow backend.


In [2]:
def creat_model(batch_size,Type):
    
    rnn_units = 12
    n_notes = 30

    input_A = keras.layers.Input(shape=(None,31), name="input_A") # (15, 20, 31)
    input_B = keras.layers.Input(shape=(1,31), name="input_B") # (15, 1, 31)

    hidden1 = keras.layers.LSTM(rnn_units, return_sequences=True)(input_A) # (15,20,12)
    hidden2 = keras.layers.LSTM(rnn_units, return_sequences=True)(hidden1) #(15,20,12)
    
    output_RNN = keras.layers.Dense(n_notes, activation='softmax', name = 'output_RNN')(hidden2) #(15,20,30)

    e = keras.layers.Dense(1, activation='tanh')(hidden2) #(15,20,1)
    e = keras.layers.Reshape([-1])(e) #(15,20)

    alpha = keras.layers.Activation('softmax')(e) #(15,20)
    c = keras.layers.Permute([2, 1])(keras.layers.RepeatVector(rnn_units)(alpha)) #(15,20,12)
    c = keras.layers.Multiply()([hidden2, c]) #(15,20,12)
    c = keras.layers.Lambda(lambda xin: K.sum(xin, axis=1), output_shape=(rnn_units,))(c) #(15,12)

    output_A = keras.layers.Dense(n_notes, activation = 'softmax', name = 'output_A')(c) #(15,30) (0, cantus)
    reshape = tf.reshape(output_A,[-1,1,30]) # (15, 1, 30)
    if Type == "_0_1":
        zero = tf.reshape(tf.zeros(batch_size),[batch_size,1,1])
        input_output_A = tf.concat((reshape,zero),2) # (15, 1, 31)  (0, cantus)
    elif Type == "_1_0":
        one = tf.reshape(tf.ones(batch_size),[batch_size,1,1])
        input_output_A = tf.concat((reshape,one),2) # (15, 1, 31)  (0, cantus)

    aux1 = keras.layers.SimpleRNN(rnn_units)(input_B, initial_state=c) #(15,12)
    aux2 = keras.layers.SimpleRNN(rnn_units)(input_output_A, initial_state=c) #(15,12)
    output_B1 = keras.layers.Dense(n_notes, activation = 'softmax', name = 'output_B1')(aux1) #(15,30) (1, counter)
    output_B2 = keras.layers.Dense(n_notes, activation = 'softmax', name = 'output_B2')(aux2) #(15,30) (1, counter)

    model = keras.models.Model([input_A, input_B], [output_RNN, output_A, output_B1, output_B2])
    
    aux_model1 = keras.models.Model([input_A, input_B], output_B1)
    aux_model2 = keras.models.Model(input_A, [output_A, output_B2])
    
    sub_model1 = keras.models.Model(input_A, output_A)
    sub_model2 = keras.models.Model(input_A, output_B2)
    
    att_model = keras.models.Model([input_A, input_B], alpha)
    
    model.compile(loss=['categorical_crossentropy', 'categorical_crossentropy','categorical_crossentropy','categorical_crossentropy'],
              loss_weights=[0.1, 0.3, 0.3, 0.3],optimizer="sgd")
    aux_model1.compile(loss=['categorical_crossentropy'],
              optimizer="sgd")
    aux_model2.compile(loss=['categorical_crossentropy', 'categorical_crossentropy'], # version 1/4: 'categorical_crossentropy'
              optimizer="sgd")                                             # version 2/3: 'kl_divergence'
    sub_model1.compile(loss='categorical_crossentropy',
              optimizer="sgd")
    sub_model2.compile(loss='categorical_crossentropy',
              optimizer="sgd")
    return model,aux_model1,aux_model2,sub_model1,sub_model2,att_model

In [3]:
model_0_1_,aux_model1_0_1_,aux_model2_0_1_, sub_model1_0_1_,sub_model2_0_1_,att_model_0_1_ = creat_model(18,'_0_1')
model_1_0_,aux_model1_1_0_,aux_model2_1_0_, sub_model1_1_0_,sub_model2_1_0_,att_model_1_0_ = creat_model(18,'_1_0')

In [4]:
model_0_1_.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_A (InputLayer)            [(None, None, 31)]   0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     (None, None, 12)     2112        input_A[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, None, 12)     1200        lstm[0][0]                       
__________________________________________________________________________________________________
dense (Dense)                   (None, None, 1)      13          lstm_1[0][0]                     
_______________________________________________________________________________________

In [5]:
model_0_1_.load_weights('model_0_1_weight.h5')
model_1_0_.load_weights('model_1_0_weight.h5')

In [6]:
df = pd.read_csv('Species.csv', sep=",",index_col = 0)
n = df.shape[0]
X = tf.Variable(np.zeros((2,n*2)),dtype='int32')  # any data tensor
for i in range(0,2*n,2):
    X[0,i].assign(df['Cantus'][i/2] - 43)
    X[1,i].assign(0)
    X[0,i+1].assign(df['Counter'][i/2] - 55)
    X[1,i+1].assign(1)
X

<tf.Variable 'Variable:0' shape=(2, 20000) dtype=int32, numpy=
array([[15, 12, 23, ..., 10, 13,  9],
       [ 0,  1,  0, ...,  1,  0,  1]])>

In [7]:
depth = 30
X = tf.concat([tf.one_hot(X[0,:], depth),tf.dtypes.cast(tf.reshape(X[1,:],[20000,1]),tf.float32)],1)
X

<tf.Tensor: shape=(20000, 31), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)>

In [8]:
# observation only
X0 = tf.Variable(np.zeros((n,31)),dtype='float32')
for i in range(n):
    X0[i,:].assign(X[2*i,:])
X0

<tf.Variable 'Variable:0' shape=(10000, 31) dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>

In [9]:
train_dataset = tf.data.Dataset.from_tensor_slices(X[:18000,:])
test_dataset = tf.data.Dataset.from_tensor_slices(X[18000:,:])

In [10]:
n_steps = 20
window_length = n_steps
dataset = train_dataset.window(window_length, shift=2, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))
batch_size = 18
shuffle_dataset = dataset.shuffle(20000).batch(batch_size,drop_remainder=True)

In [11]:
i = 0
for window in shuffle_dataset:
    i = i + 1
    print(window.shape)
    print(window)
    
print(i)

(18, 20, 31)
tf.Tensor(
[[[0. 1. 0. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]]

 [[0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 1. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ..

tf.Tensor(
[[[0. 0. 0. ... 1. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]]]

(18, 20, 31)
tf.Tensor(
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 1.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 0. 0. 1.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 1. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ..

(18, 20, 31)
tf.Tensor(
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 1. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 1. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ..

(18, 20, 31)
tf.Tensor(
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]]

 [[0. 0. 0. ... 1. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 1. 0.]
  ...
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 1. 0.]
  ...
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ..

(18, 20, 31)
tf.Tensor(
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 1. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 1. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 1. ... 0. 0. 1.]]

 [[0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 1. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ..

In [12]:
# version 1
# actively choose my action for desired results
# adapt my mind to the true observation
# experiment on the environment without cost
# minimize under categorical cross-entropy

#step = 2
#training_times = 1;
counter = 0;
loss_0_1_ = 0
X0 = np.zeros((batch_size,10,31))
X1 = np.zeros((batch_size,10,31))

#for n in range(2,step+1):
    #for times in range(training_times):
for window in shuffle_dataset: #(18,20,31)
    initial = window


    for s in range(5): # n step forward
        counter = counter + 1

        for g in range(10):
            X0[:,g,:] = initial[:,2*g,:]
            X1[:,g,:] = initial[:,2*g+1,:]

        output_A_0_1_ = sub_model1_0_1_.predict(X0) #(18,30) # my prediction about observations based on past observations


        target_0_1_1 = np.zeros((batch_size,30),dtype= 'float32') # my action

        for i in range(batch_size):
            #divergence = np.zeros((30,))
            fitness = np.zeros((30,))

            X1_slice = X1[i:i+1,:,:] #(1,10,31)
            for j in range(30):
                one_hot = tf.reshape(tf.one_hot(j, 30),[1,1,30])
                out_0_1_1 = np.concatenate((one_hot,[[[1]]]),2) #(1,1,31) # my action

                y_pred_B_1_0_ = aux_model1_1_0_.predict((X1_slice,out_0_1_1)) # my action on environment; (1,30) observation
                the_obs = np.where(y_pred_B_1_0_ == np.max(y_pred_B_1_0_))[0][0]
                fitness[j] = output_A_0_1_[i,the_obs]


                #divergence[j] = kl(output_A_0_1_[i:i+1,:], y_pred_B_1_0_).numpy() 
                # KL divergence between desired observation distribution and observation distribution under each action

            #the_j = np.where(divergence == np.min(divergence))[0][0]
            the_j = np.where(fitness == np.max(fitness))[0][0]
            one_hot = tf.reshape(tf.one_hot(the_j, 30),[1,30])
            target_0_1_1[i,:] = one_hot #(18,30) # desired action

        reshape = tf.reshape(target_0_1_1,[batch_size,1,30])
        one = tf.reshape(tf.ones(batch_size),[batch_size,1,1])
        input_action = tf.concat((reshape,one),2) # (18, 1, 31)  (1, counter)

        y_pred_B_1_0_ = aux_model1_1_0_.predict((X1, input_action)) # my action on environment; (18,30) real observation

        out_1_0_0 = np.zeros([batch_size,1,31],dtype= 'float32') #(18,1,31)
        for i in range(batch_size):
            pred_B_1_0_ = np.where(y_pred_B_1_0_[i,:] == np.max(y_pred_B_1_0_[i,:]))[0][0]
            #pred_B_1_0_ = np.random.choice(30, 1, p=y_pred_B_1_0_[i,:])[0]
            
            one_hot = tf.reshape(tf.one_hot(pred_B_1_0_, 30),[1,1,30])
            out_1_0_0[i:i+1,:,:] = np.concatenate((one_hot,[[[0]]]),2) #(18,1,31)

        target_1_0_0 = np.reshape(out_1_0_0,[batch_size,31])[:,:-1] #(18,30)

        History_0_1_ = aux_model2_0_1_.fit(X0,[target_1_0_0,target_0_1_1],verbose = 0)
        loss_0_1_ = loss_0_1_ + History_0_1_.history.get('loss')[0]

        if counter%10 == 0:
            print('model_0_1 - loss: ', loss_0_1_/10)
            loss_0_1_ = 0

        initial = tf.concat((initial[:,2:,:],out_1_0_0,input_action),1)


model_0_1 - loss:  9.351715469360352
model_0_1 - loss:  4.843849492073059
model_0_1 - loss:  3.7386590242385864
model_0_1 - loss:  3.5525678634643554
model_0_1 - loss:  2.3107054948806764
model_0_1 - loss:  2.412796342372894
model_0_1 - loss:  2.4043717861175535
model_0_1 - loss:  2.0347583651542664
model_0_1 - loss:  2.1098388195037843
model_0_1 - loss:  2.0212865114212035
model_0_1 - loss:  1.7495664358139038
model_0_1 - loss:  1.8748973250389098
model_0_1 - loss:  1.8056414365768432
model_0_1 - loss:  1.5857514798641206
model_0_1 - loss:  1.9938788950443267
model_0_1 - loss:  1.6728621125221252
model_0_1 - loss:  1.462822639942169
model_0_1 - loss:  1.5583568692207337
model_0_1 - loss:  1.4635963916778565
model_0_1 - loss:  1.3907389700412751
model_0_1 - loss:  1.2961697041988374
model_0_1 - loss:  1.1827660501003265
model_0_1 - loss:  1.2101175665855408
model_0_1 - loss:  0.9899305284023285
model_0_1 - loss:  1.351286107301712
model_0_1 - loss:  1.244774091243744
model_0_1 - loss: 

model_0_1 - loss:  0.5386095106601715
model_0_1 - loss:  0.5160148024559021
model_0_1 - loss:  0.5056201964616776
model_0_1 - loss:  0.5900058716535568
model_0_1 - loss:  0.4114418089389801
model_0_1 - loss:  0.6035750418901443
model_0_1 - loss:  0.7627769380807876
model_0_1 - loss:  0.53238804936409
model_0_1 - loss:  0.7342034697532653
model_0_1 - loss:  0.7065094739198685
model_0_1 - loss:  0.682509483397007
model_0_1 - loss:  0.5499361023306847
model_0_1 - loss:  0.5361863359808922
model_0_1 - loss:  0.5137044161558151
model_0_1 - loss:  0.6184627458453178
model_0_1 - loss:  0.6506248474121094
model_0_1 - loss:  0.6623347342014313
model_0_1 - loss:  0.5781933546066285
model_0_1 - loss:  0.4334082335233688
model_0_1 - loss:  0.5902794018387795
model_0_1 - loss:  0.5018248945474625
model_0_1 - loss:  0.5954263627529144
model_0_1 - loss:  0.4673618257045746
model_0_1 - loss:  0.5921096682548523
model_0_1 - loss:  0.5600578725337982
model_0_1 - loss:  0.49543851912021636
model_0_1 - lo

In [14]:
model_0_1_.save_weights('version1_model_0_1_step5_train1.h5')

In [97]:
output_A_0_1_[0]/np.max(output_A_0_1_[0])

array([8.9746454e-06, 4.2734380e-05, 2.4669158e-05, 2.4664655e-04,
       2.8676350e-04, 7.9257647e-04, 5.4671714e-04, 6.0456712e-04,
       7.9528970e-04, 1.0000000e+00, 9.5707783e-04, 7.3531724e-04,
       6.8719318e-04, 4.7545222e-04, 2.7620335e-04, 2.3904252e-04,
       1.1774610e-04, 8.2293838e-05, 4.9320228e-05, 3.2051565e-05,
       2.3811319e-05, 1.1937043e-05, 5.5950145e-06, 5.1744687e-06,
       1.9843476e-06, 1.1667925e-06, 3.1466648e-06, 2.7962588e-07,
       3.6064182e-07, 9.6619033e-07], dtype=float32)

In [98]:
target_1_0_0[0]/np.max(target_1_0_0[0])

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [99]:
y_pred_B_1_0_[0]/np.max(y_pred_B_1_0_[0])

array([2.4443045e-03, 4.9624285e-03, 2.6876172e-03, 6.8508066e-02,
       4.4658791e-02, 6.2413488e-02, 3.6145829e-02, 1.1956453e-02,
       3.8950604e-01, 1.0000000e+00, 3.4666717e-02, 2.3167416e-02,
       1.8443923e-02, 3.4438509e-02, 9.6583385e-03, 9.7137853e-04,
       2.5369485e-03, 8.5343653e-04, 3.1276012e-04, 5.6065604e-05,
       8.7783148e-05, 3.9401529e-03, 1.6959659e-04, 2.2310599e-04,
       5.5053103e-04, 3.0717679e-04, 2.9761426e-03, 1.3549257e-03,
       2.6494593e-03, 1.0095654e-02], dtype=float32)

In [100]:
target_0_1_1[0]/np.max(target_0_1_1[0])

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [49]:
# version 2
# actively choose my action for desired results
# adapt my mind to the true observation
# cannot experiment on the environment
# minimize under KL divergence

#step = 1
#training_times = 1;
counter = 0;
loss_0_1_ = 0
X0 = np.zeros((batch_size,10,31))
X1 = np.zeros((batch_size,10,31))

#for n in range(1,step+1):
    #for times in range(training_times):
for window in shuffle_dataset: #(18,20,31)
    initial = window

    for s in range(10): # n step forward
        counter = counter + 1

        for g in range(10):
            X0[:,g,:] = initial[:,2*g,:]
            X1[:,g,:] = initial[:,2*g+1,:]

        [output_A_0_1_, output_B2_0_1_] = aux_model2_0_1_.predict(X0) 
        #(18,30) # my prediction about observations based on past observations; the action I choose

        input_action = np.zeros([batch_size,1,31],dtype= 'float32') #(18,1,31)
        for i in range(batch_size):
            #pred_B_0_1_ = np.where(output_B2_0_1_[i,:] == np.max(output_B2_0_1_[i,:]))[0][0]
            pred_B_0_1_ = np.random.choice(30, 1, p=output_B2_0_1_[i,:])[0]
            
            one_hot = tf.reshape(tf.one_hot(pred_B_0_1_, 30),[1,1,30])
            input_action[i:i+1,:,:] = np.concatenate((one_hot,[[[1]]]),2) #(18,1,31)


        y_pred_B_1_0_ = aux_model1_1_0_.predict((X1,input_action)) # my action on environment; (18,30) real observation

        out_1_0_0 = np.zeros([batch_size,1,31],dtype= 'float32') #(18,1,31)
        action_distribution = np.zeros([batch_size,1,31],dtype= 'float32') #(18,1,31)
        for i in range(batch_size):
            #pred_B_1_0_ = np.where(y_pred_B_1_0_[i,:] == np.max(y_pred_B_1_0_[i,:]))[0][0]
            pred_B_1_0_ = np.random.choice(30, 1, p=y_pred_B_1_0_[i,:])[0]
            
            one_hot = tf.reshape(tf.one_hot(pred_B_1_0_, 30),[1,1,30])
            out_1_0_0[i:i+1,:,:] = np.concatenate((one_hot,[[[0]]]),2) #(18,1,31)

            # process my predicted observation
            normalize = output_A_0_1_[i,:]/np.max(output_A_0_1_[i,:]) #(30,)
            match = normalize[pred_B_class_max_1_0_] # range (0,1]

            pred_B_class_max_0_1_ = np.where(output_B2_0_1_[i,:] == np.max(output_B2_0_1_[i,:]))[0][0] # my action
            action_fit = np.ones(30,)*(1-match)
            action_fit[pred_B_class_max_0_1_] = match

            action_softmax = action_fit/action_fit.sum(axis=0) # (30,)
            reshape = tf.reshape(action_softmax,[1,1,30])
            action_distribution[i:i+1,:,:] = np.concatenate((reshape,[[[1]]]),2) #(18,1,31)

        target_1_0_0 = np.reshape(out_1_0_0,[batch_size,31])[:,:-1] #(18,30) real observation
        target_0_1_1 = np.reshape(action_distribution,[batch_size,31])[:,:-1] #(18,30) desired action distribution

        History_0_1_ = aux_model2_0_1_.fit(X0,[target_1_0_0,target_0_1_1],verbose = 0)
        loss_0_1_ = loss_0_1_ + History_0_1_.history.get('loss')[0]

        if counter%50 == 0:
            print('model_0_1 - loss: ', loss_0_1_/50)
            loss_0_1_ = 0

        initial = tf.concat((initial[:,2:,:],out_1_0_0,input_action),1)


model_0_1 - loss:  4.263684959411621
model_0_1 - loss:  4.020083475112915
model_0_1 - loss:  3.980966401100159
model_0_1 - loss:  3.931880078315735
model_0_1 - loss:  3.7673660373687743
model_0_1 - loss:  3.6153246784210205
model_0_1 - loss:  3.4659902811050416
model_0_1 - loss:  3.154281930923462
model_0_1 - loss:  3.350224094390869
model_0_1 - loss:  3.267438383102417
model_0_1 - loss:  3.039626746177673
model_0_1 - loss:  2.9858588552474976
model_0_1 - loss:  2.9463092374801634
model_0_1 - loss:  3.110018630027771
model_0_1 - loss:  2.975547962188721
model_0_1 - loss:  2.9879052686691283
model_0_1 - loss:  2.9274505376815796
model_0_1 - loss:  3.022915549278259
model_0_1 - loss:  3.0745911836624145
model_0_1 - loss:  3.184143776893616
model_0_1 - loss:  3.1621369886398316
model_0_1 - loss:  3.1536965417861937
model_0_1 - loss:  3.1365860652923585
model_0_1 - loss:  3.102460560798645
model_0_1 - loss:  2.9942395877838135
model_0_1 - loss:  3.126015615463257
model_0_1 - loss:  3.12144

In [50]:
model_0_1_.save_weights('version2_model_0_1_step10_train1_random.h5')

In [28]:
# version 3
# niche construction
# with one sample delay
# minimize under KL divergence


#step = 2
#training_times = 3;
counter = 0;
loss_0_1_ = 0
loss_1_0_ = 0
X0 = np.zeros((batch_size,10,31))
X1 = np.zeros((batch_size,10,31))

#for n in range(1,step+1):
    #for times in range(training_times):
for window in shuffle_dataset: #(18,20,31)
    initial = window


    for s in range(5): # n step forward
        counter = counter + 1

        for g in range(10):
            X0[:,g,:] = initial[:,2*g,:]
            X1[:,g,:] = initial[:,2*g+1,:] #(18,10,31)

        [output_A_0_1_, output_B2_0_1_] = aux_model2_0_1_.predict(X0) 
        #(18,30) # my prediction about observations based on past observations; the action I choose

        input_action = np.zeros([batch_size,1,31],dtype= 'float32') #(18,1,31)
        pred_B_0_1_ = np.zeros([batch_size,],dtype= 'int8')
        for i in range(batch_size):
            #pred_B_0_1_[i] = np.where(output_B2_0_1_[i,:] == np.max(output_B2_0_1_[i,:]))[0][0]
            pred_B_0_1_[i] = np.random.choice(30, 1, p=output_B2_0_1_[i,:])[0]
            
            one_hot = tf.reshape(tf.one_hot(pred_B_0_1_[i], 30),[1,1,30])
            input_action[i:i+1,:,:] = np.concatenate((one_hot,[[[1]]]),2) #(18,1,31)

        X1 = np.concatenate((X1[:,1:,:],input_action),axis=1)

        [y_pred_A_1_0_,y_pred_B_1_0_] = aux_model2_1_0_.predict(X1)  
        #(18,30) # environment's prediction about t+1 action based on past actions; the observation the environment choose

        out_1_0_0 = np.zeros([batch_size,1,31],dtype= 'float32') #(18,1,31)
        action_distribution = np.zeros([batch_size,1,31],dtype= 'float32') #(18,1,31)
        
        pred_B_1_0_ = np.zeros([batch_size,],dtype= 'int8')
        for i in range(batch_size):
            #pred_B_1_0_[i] = np.where(y_pred_B_1_0_[i,:] == np.max(y_pred_B_1_0_[i,:]))[0][0]
            pred_B_1_0_[i] = np.random.choice(30, 1, p=y_pred_B_1_0_[i,:])[0]
            
            one_hot = tf.reshape(tf.one_hot(pred_B_1_0_[i], 30),[1,1,30])
            out_1_0_0[i:i+1,:,:] = np.concatenate((one_hot,[[[0]]]),2) #(18,1,31)

            # process my predicted observation
            normalize = output_A_0_1_[i,:]/np.max(output_A_0_1_[i,:]) #(30,)
            match = normalize[pred_B_1_0_[i]] # range (0,1]

            action_fit = np.ones(30,)*(1-match)
            action_fit[pred_B_0_1_[i]] = match

            action_softmax = action_fit/action_fit.sum(axis=0) # (30,)
            reshape = tf.reshape(action_softmax,[1,1,30])
            action_distribution[i:i+1,:,:] = np.concatenate((reshape,[[[1]]]),2) #(18,1,31)

        target_1_0_0 = np.reshape(out_1_0_0,[batch_size,31])[:,:-1] #(18,30) real observation
        target_0_1_1 = np.reshape(action_distribution,[batch_size,31])[:,:-1] #(18,30) desired action distribution

        History_0_1_ = aux_model2_0_1_.fit(X0,[target_1_0_0,target_0_1_1],verbose = 0)
        loss_0_1_ = loss_0_1_ + History_0_1_.history.get('loss')[0]

        X0 = np.concatenate((X0[:,1:,:],out_1_0_0),axis=1)

        [output_A_0_1_delay, output_B2_0_1_delay] = aux_model2_0_1_.predict(X0) 
        #(18,30) # my prediction about t+1 observation based on past observations; the t+1 action I choose

        t1_action = np.zeros([batch_size,1,31],dtype= 'float32') #(18,1,31)
        observation_distribution = np.zeros([batch_size,1,31],dtype= 'float32') #(18,1,31)
        for i in range(batch_size):
            #pred_B_0_1_ = np.where(output_B2_0_1_delay[i,:] == np.max(output_B2_0_1_delay[i,:]))[0][0]
            pred_B_0_1_ = np.random.choice(30, 1, p=output_B2_0_1_delay[i,:])[0]
            
            one_hot = tf.reshape(tf.one_hot(pred_B_0_1_, 30),[1,1,30])
            t1_action[i:i+1,:,:] = np.concatenate((one_hot,[[[1]]]),2) #(18,1,31)

            # process environment's predicted t+1 action
            normalize = y_pred_A_1_0_[i,:]/np.max(y_pred_A_1_0_[i,:]) #(30,)
            match = normalize[pred_B_0_1_] # range (0,1]

            observation_fit = np.ones(30,)*(1-match)
            observation_fit[pred_B_1_0_[i]] = match

            observation_softmax = observation_fit/observation_fit.sum(axis=0) # (30,)
            reshape = tf.reshape(observation_softmax,[1,1,30])
            observation_distribution[i:i+1,:,:] = np.concatenate((reshape,[[[0]]]),2) #(18,1,31)

        target_t1_action = np.reshape(t1_action,[batch_size,31])[:,:-1] #(18,30) t+1 action
        target_0_1_0 = np.reshape(observation_distribution,[batch_size,31])[:,:-1] #(18,30) desired observation distribution


        History_1_0_ = aux_model2_1_0_.fit(X1,[target_t1_action,target_0_1_0],verbose = 0)
        loss_1_0_ = loss_1_0_ + History_1_0_.history.get('loss')[0]

        if counter%50 == 0:
            print('model_0_1 - loss: ', loss_0_1_/50)
            print('model_1_0 - loss: ', loss_1_0_/50)
            loss_0_1_ = 0
            loss_1_0_ = 0

        initial = tf.concat((initial[:,2:,:],out_1_0_0,input_action),1)


model_0_1 - loss:  4.338302435874939
model_1_0 - loss:  4.505087785720825
model_0_1 - loss:  3.767235860824585
model_1_0 - loss:  4.130347385406494
model_0_1 - loss:  3.7760346794128417
model_1_0 - loss:  3.857211961746216
model_0_1 - loss:  3.860733585357666
model_1_0 - loss:  3.7969043493270873
model_0_1 - loss:  3.861966972351074
model_1_0 - loss:  3.7682492923736572
model_0_1 - loss:  3.8368313026428225
model_1_0 - loss:  3.707562804222107
model_0_1 - loss:  3.815206542015076
model_1_0 - loss:  3.7198505353927613
model_0_1 - loss:  3.7869335079193114
model_1_0 - loss:  3.6655563354492187
model_0_1 - loss:  3.7651600456237793
model_1_0 - loss:  3.6671522092819213
model_0_1 - loss:  3.804746217727661
model_1_0 - loss:  3.6987478351593017
model_0_1 - loss:  3.765591177940369
model_1_0 - loss:  3.6968717432022093
model_0_1 - loss:  3.7883744478225707
model_1_0 - loss:  3.6569949197769165
model_0_1 - loss:  3.7238007545471192
model_1_0 - loss:  3.718032522201538
model_0_1 - loss:  3.742

In [29]:
model_0_1_.save_weights('version3_model_0_1_step5_train1_random.h5')
model_1_0_.save_weights('version3_model_1_0_step5_train1_random.h5')

In [18]:
np.zeros([batch_size,],dtype= 'int8')

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int8)

In [84]:
output_B2_0_1_[0]/np.max(output_B2_0_1_[0])

array([4.0871708e-04, 4.7610627e-04, 2.8659619e-04, 6.8463845e-04,
       4.7934189e-04, 8.9377179e-05, 2.0216106e-04, 4.5165286e-04,
       4.3713141e-04, 4.4341118e-04, 4.0293369e-04, 2.6984088e-04,
       3.2243255e-04, 1.2602448e-04, 5.0188438e-04, 1.3539498e-04,
       3.3401299e-04, 3.8634552e-04, 1.0000000e+00, 8.0356462e-05,
       3.8293592e-04, 5.6279293e-04, 3.5987742e-04, 3.7070003e-04,
       3.1857460e-04, 3.7304492e-04, 2.5969557e-04, 2.4591445e-04,
       1.7746488e-04, 2.6455033e-04], dtype=float32)

In [79]:
y_pred_B_1_0_[0]/np.max(y_pred_B_1_0_[0])

array([2.0141246e-04, 1.0000000e+00, 5.7024916e-04, 1.4765107e-04,
       2.3325806e-04, 2.1459557e-04, 2.7651805e-04, 5.0987152e-04,
       2.9895338e-04, 2.2936355e-04, 2.0497738e-04, 3.4570164e-04,
       5.6489435e-04, 1.5137323e-04, 2.7731454e-04, 4.1434279e-04,
       1.3644088e-04, 6.3002924e-04, 2.1480092e-04, 2.2336657e-04,
       2.9595185e-04, 1.8197922e-04, 3.9272572e-04, 2.8461087e-04,
       3.4895405e-04, 1.1837628e-04, 3.5057447e-04, 4.4024634e-04,
       2.3919395e-04, 2.2173804e-04], dtype=float32)

In [21]:
# version 4
# synchronization
# access to maximum hidden states
# two-side adaption


#step = 2
#training_times = 3;
counter = 0;
loss_0_1_ = 0
loss_1_0_ = 0


#for n in range(1,step+1):
    #for times in range(training_times):
for window in shuffle_dataset: #(18,20,31)
    initial = window


    for s in range(2): # n step forward
        counter = counter + 1


        [output_A_0_1_, output_B2_0_1_] = aux_model2_0_1_.predict(initial) 
        #(18,30) # my prediction about observations based on past observations; the action I choose

        out_0_1_0 = np.zeros([batch_size,1,31],dtype= 'float32') #(18,1,31)
        out_0_1_1 = np.zeros([batch_size,1,31],dtype= 'float32') #(18,1,31)
        for i in range(batch_size):
            pred_A_class_max_0_1_ = np.where(output_A_0_1_[i,:] == np.max(output_A_0_1_[i,:]))[0][0]
            
            one_hot = tf.reshape(tf.one_hot(pred_A_class_max_0_1_, 30),[1,1,30])
            out_0_1_0[i:i+1,:,:] = np.concatenate((one_hot,[[[0]]]),2) #(18,1,31)

            #pred_B_0_1_ = np.where(output_B2_0_1_[i,:] == np.max(output_B2_0_1_[i,:]))[0][0]
            pred_B_0_1_ = np.random.choice(30, 1, p=output_B2_0_1_[i,:])[0]
            
            one_hot = tf.reshape(tf.one_hot(pred_B_0_1_, 30),[1,1,30])
            out_0_1_1[i:i+1,:,:] = np.concatenate((one_hot,[[[1]]]),2) #(18,1,31)


        [y_pred_A_1_0_,y_pred_B_1_0_] = aux_model2_1_0_.predict(initial)  
        #(18,30) # environment's prediction about t+1 action based on past actions; the observation the environment choose

        out_1_0_1 = np.zeros([batch_size,1,31],dtype= 'float32') #(18,1,31)
        out_1_0_0 = np.zeros([batch_size,1,31],dtype= 'float32') #(18,1,31)
        for i in range(batch_size):
            pred_A_class_max_1_0_ = np.where(y_pred_A_1_0_[i,:] == np.max(y_pred_A_1_0_[i,:]))[0][0]
            one_hot = tf.reshape(tf.one_hot(pred_A_class_max_1_0_, 30),[1,1,30])
            out_1_0_1[i:i+1,:,:] = np.concatenate((one_hot,[[[1]]]),2) #(18,1,31)

            #pred_B_1_0_ = np.where(y_pred_B_1_0_[i,:] == np.max(y_pred_B_1_0_[i,:]))[0][0]
            pred_B_1_0_ = np.random.choice(30, 1, p=y_pred_B_1_0_[i,:])[0]
            
            one_hot = tf.reshape(tf.one_hot(pred_B_1_0_, 30),[1,1,30])
            out_1_0_0[i:i+1,:,:] = np.concatenate((one_hot,[[[0]]]),2) #(18,1,31)

        target_0_1_0 = np.reshape(out_0_1_0,[batch_size,31])[:,:-1] #(18,30) pred observation
        target_0_1_1 = np.reshape(out_0_1_1,[batch_size,31])[:,:-1] #(18,30) real action    
        target_1_0_1 = np.reshape(out_1_0_1,[batch_size,31])[:,:-1] #(18,30) pred action
        target_1_0_0 = np.reshape(out_1_0_0,[batch_size,31])[:,:-1] #(18,30) real observation

        History_0_1_ = aux_model2_0_1_.fit(initial,[target_1_0_0,target_1_0_1],verbose = 0)
        loss_0_1_ = loss_0_1_ + History_0_1_.history.get('loss')[0]

        History_1_0_ = aux_model2_1_0_.fit(initial,[target_0_1_1,target_0_1_0],verbose = 0)
        loss_1_0_ = loss_1_0_ + History_1_0_.history.get('loss')[0]

        if counter%50 == 0:
            print('model_0_1 - loss: ', loss_0_1_/50)
            print('model_1_0 - loss: ', loss_1_0_/50)
            loss_0_1_ = 0
            loss_1_0_ = 0

        initial = tf.concat((initial[:,2:,:],out_1_0_0,out_0_1_1),1)


model_0_1 - loss:  4.547919921875
model_1_0 - loss:  4.5500581121444705
model_0_1 - loss:  4.02534279346466
model_1_0 - loss:  4.047863278388977
model_0_1 - loss:  3.371981673240662
model_1_0 - loss:  3.359418730735779
model_0_1 - loss:  2.1277092909812927
model_1_0 - loss:  2.0105354404449463
model_0_1 - loss:  0.8642736572027206
model_1_0 - loss:  0.7940643066167832
model_0_1 - loss:  0.581370800435543
model_1_0 - loss:  0.46619615107774737
model_0_1 - loss:  0.42148600190877916
model_1_0 - loss:  0.37759469836950305
model_0_1 - loss:  0.2559310168027878
model_1_0 - loss:  0.24096179828047753
model_0_1 - loss:  0.25262033715844157
model_1_0 - loss:  0.21963717728853227
model_0_1 - loss:  0.210238346606493
model_1_0 - loss:  0.2650184997916222
model_0_1 - loss:  0.2050800822675228
model_1_0 - loss:  0.18676299214363098
model_0_1 - loss:  0.14398015588521956
model_1_0 - loss:  0.18469009466469288
model_0_1 - loss:  0.12026582911610603
model_1_0 - loss:  0.1740585593134165
model_0_1 - l

In [22]:
model_0_1_.save_weights('version4_model_0_1_step2_train1_random.h5')
model_1_0_.save_weights('version4_model_1_0_step2_train1_random.h5')

In [57]:
# version 5
# synchronization
# access to random chosen hidden states
# two-side adaption


#step = 2
#training_times = 3;
counter = 0;
loss_0_1_ = 0
loss_1_0_ = 0


#for n in range(1,step+1):
    #for times in range(training_times):
for window in shuffle_dataset: #(18,20,31)
    initial = window


    for s in range(5): # n step forward
        counter = counter + 1


        [output_A_0_1_, output_B2_0_1_] = aux_model2_0_1_.predict(initial) 
        #(18,30) # my prediction about observations based on past observations; the action I choose

        out_0_1_0 = np.zeros([batch_size,1,31],dtype= 'float32') #(18,1,31)
        out_0_1_1 = np.zeros([batch_size,1,31],dtype= 'float32') #(18,1,31)
        for i in range(batch_size):
            pred_A_class_random_0_1_ = np.random.choice(30, 1, p=output_A_0_1_[i,:])[0]
            
            one_hot = tf.reshape(tf.one_hot(pred_A_class_random_0_1_, 30),[1,1,30])
            out_0_1_0[i:i+1,:,:] = np.concatenate((one_hot,[[[0]]]),2) #(18,1,31)

            #pred_B_0_1_ = np.where(output_B2_0_1_[i,:] == np.max(output_B2_0_1_[i,:]))[0][0]
            pred_B_0_1_ = np.random.choice(30, 1, p=output_B2_0_1_[i,:])[0]
            
            one_hot = tf.reshape(tf.one_hot(pred_B_0_1_, 30),[1,1,30])
            out_0_1_1[i:i+1,:,:] = np.concatenate((one_hot,[[[1]]]),2) #(18,1,31)


        [y_pred_A_1_0_,y_pred_B_1_0_] = aux_model2_1_0_.predict(initial)  
        #(18,30) # environment's prediction about t+1 action based on past actions; the observation the environment choose

        out_1_0_1 = np.zeros([batch_size,1,31],dtype= 'float32') #(18,1,31)
        out_1_0_0 = np.zeros([batch_size,1,31],dtype= 'float32') #(18,1,31)
        for i in range(batch_size):
            pred_A_class_random_1_0_ = np.random.choice(30, 1, p=y_pred_A_1_0_[i,:])[0]
            
            one_hot = tf.reshape(tf.one_hot(pred_A_class_random_1_0_, 30),[1,1,30]) 
            out_1_0_1[i:i+1,:,:] = np.concatenate((one_hot,[[[1]]]),2) #(18,1,31)

            #pred_B_1_0_ = np.where(y_pred_B_1_0_[i,:] == np.max(y_pred_B_1_0_[i,:]))[0][0]
            pred_B_1_0_ = np.random.choice(30, 1, p=y_pred_B_1_0_[i,:])[0]
            
            one_hot = tf.reshape(tf.one_hot(pred_B_1_0_, 30),[1,1,30])
            out_1_0_0[i:i+1,:,:] = np.concatenate((one_hot,[[[0]]]),2) #(18,1,31)

        target_0_1_0 = np.reshape(out_0_1_0,[batch_size,31])[:,:-1] #(18,30) pred observation
        target_0_1_1 = np.reshape(out_0_1_1,[batch_size,31])[:,:-1] #(18,30) real action    
        target_1_0_1 = np.reshape(out_1_0_1,[batch_size,31])[:,:-1] #(18,30) pred action
        target_1_0_0 = np.reshape(out_1_0_0,[batch_size,31])[:,:-1] #(18,30) real observation

        History_0_1_ = aux_model2_0_1_.fit(initial,[target_1_0_0,target_1_0_1],verbose = 0)
        loss_0_1_ = loss_0_1_ + History_0_1_.history.get('loss')[0]

        History_1_0_ = aux_model2_1_0_.fit(initial,[target_0_1_1,target_0_1_0],verbose = 0)
        loss_1_0_ = loss_1_0_ + History_1_0_.history.get('loss')[0]

        if counter%50 == 0:
            print('model_0_1 - loss: ', loss_0_1_/50)
            print('model_1_0 - loss: ', loss_1_0_/50)
            loss_0_1_ = 0
            loss_1_0_ = 0

        initial = tf.concat((initial[:,2:,:],out_1_0_0,out_0_1_1),1)


model_0_1 - loss:  5.397593030929565
model_1_0 - loss:  5.497243728637695
model_0_1 - loss:  5.355459938049316
model_1_0 - loss:  5.290720148086548
model_0_1 - loss:  5.258261346817017
model_1_0 - loss:  5.2667842197418215
model_0_1 - loss:  5.259473896026611
model_1_0 - loss:  5.281960592269898
model_0_1 - loss:  5.229804239273071
model_1_0 - loss:  5.229663772583008
model_0_1 - loss:  5.278004236221314
model_1_0 - loss:  5.266049461364746
model_0_1 - loss:  5.273361511230469
model_1_0 - loss:  5.17803837776184
model_0_1 - loss:  5.231135416030884
model_1_0 - loss:  5.242010688781738
model_0_1 - loss:  5.149896049499512
model_1_0 - loss:  5.185154714584351
model_0_1 - loss:  5.2235010051727295
model_1_0 - loss:  5.11287974357605
model_0_1 - loss:  5.154891757965088
model_1_0 - loss:  5.1620496654510495
model_0_1 - loss:  5.181703329086304
model_1_0 - loss:  5.138739967346192
model_0_1 - loss:  5.208856029510498
model_1_0 - loss:  5.140689506530761
model_0_1 - loss:  5.255821294784546


In [58]:
model_0_1_.save_weights('version5_model_0_1_step5_train1_random.h5')
model_1_0_.save_weights('version5_model_1_0_step5_train1_random.h5')

In [83]:
# version 6
# synchronization
# access to hidden state distribution
# two-side adaption


#step = 2
#training_times = 3;
counter = 0;
loss_0_1_ = 0
loss_1_0_ = 0


#for n in range(1,step+1):
    #for times in range(training_times):
for window in shuffle_dataset: #(18,20,31)
    initial = window


    for s in range(10): # n step forward
        counter = counter + 1


        [output_A_0_1_, output_B2_0_1_] = aux_model2_0_1_.predict(initial) 
        #(18,30) # my prediction about observations based on past observations; the action I choose

        reshape = tf.reshape(output_A_0_1_,[batch_size,1,30])
        zero = tf.reshape(tf.zeros(batch_size),[batch_size,1,1])
        out_0_1_0 = tf.concat((reshape,zero),2)
        
        out_0_1_1 = np.zeros([batch_size,1,31],dtype= 'float32') #(18,1,31)
        for i in range(batch_size):

            #pred_B_0_1_ = np.where(output_B2_0_1_[i,:] == np.max(output_B2_0_1_[i,:]))[0][0]
            pred_B_0_1_ = np.random.choice(30, 1, p=output_B2_0_1_[i,:])[0]
            
            one_hot = tf.reshape(tf.one_hot(pred_B_0_1_, 30),[1,1,30])
            out_0_1_1[i:i+1,:,:] = np.concatenate((one_hot,[[[1]]]),2) #(18,1,31)


        [y_pred_A_1_0_,y_pred_B_1_0_] = aux_model2_1_0_.predict(initial)  
        #(18,30) # environment's prediction about t+1 action based on past actions; the observation the environment choose

        reshape = tf.reshape(y_pred_A_1_0_,[batch_size,1,30])
        one = tf.reshape(tf.ones(batch_size),[batch_size,1,1])
        out_1_0_1 = tf.concat((reshape,one),2)
        
        out_1_0_0 = np.zeros([batch_size,1,31],dtype= 'float32') #(18,1,31)
        for i in range(batch_size):
            
            #pred_B_1_0_ = np.where(y_pred_B_1_0_[i,:] == np.max(y_pred_B_1_0_[i,:]))[0][0]
            pred_B_1_0_ = np.random.choice(30, 1, p=y_pred_B_1_0_[i,:])[0]
            
            one_hot = tf.reshape(tf.one_hot(pred_B_1_0_, 30),[1,1,30])
            out_1_0_0[i:i+1,:,:] = np.concatenate((one_hot,[[[0]]]),2) #(18,1,31)

        target_0_1_0 = output_A_0_1_ #(18,30) pred observation distribution
        target_0_1_1 = np.reshape(out_0_1_1,[batch_size,31])[:,:-1] #(18,30) real action    
        target_1_0_1 = y_pred_A_1_0_ #(18,30) pred action distribution
        target_1_0_0 = np.reshape(out_1_0_0,[batch_size,31])[:,:-1] #(18,30) real observation

        History_0_1_ = aux_model2_0_1_.fit(initial,[target_1_0_0,target_1_0_1],verbose = 0)
        loss_0_1_ = loss_0_1_ + History_0_1_.history.get('loss')[0]

        History_1_0_ = aux_model2_1_0_.fit(initial,[target_0_1_1,target_0_1_0],verbose = 0)
        loss_1_0_ = loss_1_0_ + History_1_0_.history.get('loss')[0]

        if counter%50 == 0:
            print('model_0_1 - loss: ', loss_0_1_/50)
            print('model_1_0 - loss: ', loss_1_0_/50)
            loss_0_1_ = 0
            loss_1_0_ = 0

        initial = tf.concat((initial[:,2:,:],out_1_0_0,out_0_1_1),1)


model_0_1 - loss:  2.7031643486022947
model_1_0 - loss:  2.752235598564148
model_0_1 - loss:  2.672102189064026
model_1_0 - loss:  2.666750183105469
model_0_1 - loss:  2.7196799898147583
model_1_0 - loss:  2.6567387199401855
model_0_1 - loss:  2.689943370819092
model_1_0 - loss:  2.649090166091919
model_0_1 - loss:  2.643182315826416
model_1_0 - loss:  2.6402749586105347
model_0_1 - loss:  2.700678925514221
model_1_0 - loss:  2.563104248046875
model_0_1 - loss:  2.631908278465271
model_1_0 - loss:  2.587908606529236
model_0_1 - loss:  2.6648278570175172
model_1_0 - loss:  2.6435822439193726
model_0_1 - loss:  2.635305438041687
model_1_0 - loss:  2.5990817451477053
model_0_1 - loss:  2.6802539014816285
model_1_0 - loss:  2.56812931060791
model_0_1 - loss:  2.651265330314636
model_1_0 - loss:  2.5769290781021117
model_0_1 - loss:  2.6617454528808593
model_1_0 - loss:  2.48052695274353
model_0_1 - loss:  2.600682110786438
model_1_0 - loss:  2.4688250255584716
model_0_1 - loss:  2.59224324

In [84]:
model_0_1_.save_weights('version6_model_0_1_step10_train1_random.h5')
model_1_0_.save_weights('version6_model_1_0_step10_train1_random.h5')

In [18]:
# version 7
# synchronization
# no access to hidden states
# two-side adaption

#step = 2
#training_times = 3;
counter = 0;
loss_0_1_ = 0
loss_1_0_ = 0


#for n in range(1,step+1):
    #for times in range(training_times):
for window in shuffle_dataset: #(18,20,31)
    initial = window


    for s in range(5): # n step forward
        counter = counter + 1


        [output_A_0_1_, output_B2_0_1_] = aux_model2_0_1_.predict(initial) 
        #(18,30) # my prediction about observations based on past observations; the action I choose
        
        out_0_1_1 = np.zeros([batch_size,1,31],dtype= 'float32') #(18,1,31)
        for i in range(batch_size):

            #pred_B_0_1_ = np.where(output_B2_0_1_[i,:] == np.max(output_B2_0_1_[i,:]))[0][0]
            pred_B_0_1_ = np.random.choice(30, 1, p=output_B2_0_1_[i,:])[0]
            
            one_hot = tf.reshape(tf.one_hot(pred_B_0_1_, 30),[1,1,30])
            out_0_1_1[i:i+1,:,:] = np.concatenate((one_hot,[[[1]]]),2) #(18,1,31)


        [y_pred_A_1_0_,y_pred_B_1_0_] = aux_model2_1_0_.predict(initial)  
        #(18,30) # environment's prediction about t+1 action based on past actions; the observation the environment choose

        
        out_1_0_0 = np.zeros([batch_size,1,31],dtype= 'float32') #(18,1,31)
        for i in range(batch_size):
            
            #pred_B_1_0_ = np.where(y_pred_B_1_0_[i,:] == np.max(y_pred_B_1_0_[i,:]))[0][0]
            pred_B_1_0_ = np.random.choice(30, 1, p=y_pred_B_1_0_[i,:])[0]
            
            one_hot = tf.reshape(tf.one_hot(pred_B_1_0_, 30),[1,1,30])
            out_1_0_0[i:i+1,:,:] = np.concatenate((one_hot,[[[0]]]),2) #(18,1,31)

        
        target_0_1_1 = np.reshape(out_0_1_1,[batch_size,31])[:,:-1] #(18,30) real action    
        target_1_0_0 = np.reshape(out_1_0_0,[batch_size,31])[:,:-1] #(18,30) real observation

        History_0_1_ = sub_model1_0_1_.fit(initial,target_1_0_0,verbose = 0)
        loss_0_1_ = loss_0_1_ + History_0_1_.history.get('loss')[0]

        History_1_0_ = sub_model1_1_0_.fit(initial,target_0_1_1,verbose = 0)
        loss_1_0_ = loss_1_0_ + History_1_0_.history.get('loss')[0]

        if counter%50 == 0:
            print('model_0_1 - loss: ', loss_0_1_/50)
            print('model_1_0 - loss: ', loss_1_0_/50)
            loss_0_1_ = 0
            loss_1_0_ = 0

        initial = tf.concat((initial[:,2:,:],out_1_0_0,out_0_1_1),1)


model_0_1 - loss:  2.635420627593994
model_1_0 - loss:  2.5555783700942993
model_0_1 - loss:  2.6135129833221438
model_1_0 - loss:  2.383434388637543
model_0_1 - loss:  2.6206812191009523
model_1_0 - loss:  2.3110160279273986
model_0_1 - loss:  2.6088682889938353
model_1_0 - loss:  2.270932548046112
model_0_1 - loss:  2.5466840839385987
model_1_0 - loss:  2.3290332102775575
model_0_1 - loss:  2.556411175727844
model_1_0 - loss:  2.2684181118011475
model_0_1 - loss:  2.5434086179733275
model_1_0 - loss:  2.2526619362831117
model_0_1 - loss:  2.5516429233551023
model_1_0 - loss:  2.2507270073890684
model_0_1 - loss:  2.535975456237793
model_1_0 - loss:  2.2601975440979003
model_0_1 - loss:  2.5308886909484865
model_1_0 - loss:  2.2305143380165102
model_0_1 - loss:  2.586549940109253
model_1_0 - loss:  2.3038051652908327
model_0_1 - loss:  2.534792814254761
model_1_0 - loss:  2.2354836297035217
model_0_1 - loss:  2.5605647230148314
model_1_0 - loss:  2.255969879627228
model_0_1 - loss:  2

In [19]:
model_0_1_.save_weights('version7_model_0_1_step5_train1_random.h5')
model_1_0_.save_weights('version7_model_1_0_step5_train1_random.h5')

In [58]:
# version 8
# synchronization
# access to hidden state distribution
# one-side adaption

#step = 2
#training_times = 3;
counter = 0;
loss_0_1_ = 0


#for n in range(1,step+1):
    #for times in range(training_times):
for window in shuffle_dataset: #(18,20,31)
    initial = window


    for s in range(2): # n step forward
        counter = counter + 1

        [output_A_0_1_, output_B2_0_1_] = aux_model2_0_1_.predict(initial) 
        #(18,30) # my prediction about observations based on past observations; the action I choose

        reshape = tf.reshape(output_A_0_1_,[batch_size,1,30])
        zero = tf.reshape(tf.zeros(batch_size),[batch_size,1,1])
        out_0_1_0 = tf.concat((reshape,zero),2)
        
        out_0_1_1 = np.zeros([batch_size,1,31],dtype= 'float32') #(18,1,31)
        for i in range(batch_size):

            #pred_B_0_1_ = np.where(output_B2_0_1_[i,:] == np.max(output_B2_0_1_[i,:]))[0][0]
            pred_B_0_1_ = np.random.choice(30, 1, p=output_B2_0_1_[i,:])[0]
            
            one_hot = tf.reshape(tf.one_hot(pred_B_0_1_, 30),[1,1,30])
            out_0_1_1[i:i+1,:,:] = np.concatenate((one_hot,[[[1]]]),2) #(18,1,31)


        [y_pred_A_1_0_,y_pred_B_1_0_] = aux_model2_1_0_.predict(initial)  
        #(18,30) # environment's prediction about t+1 action based on past actions; the observation the environment choose

        reshape = tf.reshape(y_pred_A_1_0_,[batch_size,1,30])
        one = tf.reshape(tf.ones(batch_size),[batch_size,1,1])
        out_1_0_1 = tf.concat((reshape,one),2)
        
        out_1_0_0 = np.zeros([batch_size,1,31],dtype= 'float32') #(18,1,31)
        for i in range(batch_size):
            
            #pred_B_1_0_ = np.where(y_pred_B_1_0_[i,:] == np.max(y_pred_B_1_0_[i,:]))[0][0]
            pred_B_1_0_ = np.random.choice(30, 1, p=y_pred_B_1_0_[i,:])[0]
            
            one_hot = tf.reshape(tf.one_hot(pred_B_1_0_, 30),[1,1,30])
            out_1_0_0[i:i+1,:,:] = np.concatenate((one_hot,[[[0]]]),2) #(18,1,31)

        target_0_1_0 = output_A_0_1_ #(18,30) pred observation distribution
        target_0_1_1 = np.reshape(out_0_1_1,[batch_size,31])[:,:-1] #(18,30) real action    
        target_1_0_1 = y_pred_A_1_0_ #(18,30) pred action distribution
        target_1_0_0 = np.reshape(out_1_0_0,[batch_size,31])[:,:-1] #(18,30) real observation

        History_0_1_ = aux_model2_0_1_.fit(initial,[target_1_0_0,target_1_0_1],verbose = 0)
        loss_0_1_ = loss_0_1_ + History_0_1_.history.get('loss')[0]


        if counter%50 == 0:
            print('model_0_1 - loss: ', loss_0_1_/50)
            loss_0_1_ = 0
            
        initial = tf.concat((initial[:,2:,:],out_1_0_0,out_0_1_1),1)


model_0_1 - loss:  2.7028154706954957
model_0_1 - loss:  2.7476448392868043
model_0_1 - loss:  2.693182153701782
model_0_1 - loss:  2.718983144760132
model_0_1 - loss:  2.6878364515304565
model_0_1 - loss:  2.673534116744995
model_0_1 - loss:  2.678345513343811
model_0_1 - loss:  2.7011575222015383
model_0_1 - loss:  2.689706072807312
model_0_1 - loss:  2.678988103866577
model_0_1 - loss:  2.6676267194747925
model_0_1 - loss:  2.688626194000244
model_0_1 - loss:  2.693516755104065
model_0_1 - loss:  2.6670537185668945
model_0_1 - loss:  2.690176315307617
model_0_1 - loss:  2.725991187095642
model_0_1 - loss:  2.6623876810073854
model_0_1 - loss:  2.6613200378417967
model_0_1 - loss:  2.6797636127471924


In [59]:
model_0_1_.save_weights('version8_model_0_1_step2_train1_random.h5')

In [71]:
# version 9
# synchronization
# access to maximum hidden states
# one-side adaption


#step = 2
#training_times = 3;
counter = 0;
loss_0_1_ = 0


#for n in range(1,step+1):
    #for times in range(training_times):
for window in shuffle_dataset: #(18,20,31)
    initial = window


    for s in range(5): # n step forward
        counter = counter + 1


        [output_A_0_1_, output_B2_0_1_] = aux_model2_0_1_.predict(initial) 
        #(18,30) # my prediction about observations based on past observations; the action I choose

        out_0_1_0 = np.zeros([batch_size,1,31],dtype= 'float32') #(18,1,31)
        out_0_1_1 = np.zeros([batch_size,1,31],dtype= 'float32') #(18,1,31)
        for i in range(batch_size):
            pred_A_class_max_0_1_ = np.where(output_A_0_1_[i,:] == np.max(output_A_0_1_[i,:]))[0][0]
            
            one_hot = tf.reshape(tf.one_hot(pred_A_class_max_0_1_, 30),[1,1,30])
            out_0_1_0[i:i+1,:,:] = np.concatenate((one_hot,[[[0]]]),2) #(18,1,31)

            #pred_B_0_1_ = np.where(output_B2_0_1_[i,:] == np.max(output_B2_0_1_[i,:]))[0][0]
            pred_B_0_1_ = np.random.choice(30, 1, p=output_B2_0_1_[i,:])[0]
            
            one_hot = tf.reshape(tf.one_hot(pred_B_0_1_, 30),[1,1,30])
            out_0_1_1[i:i+1,:,:] = np.concatenate((one_hot,[[[1]]]),2) #(18,1,31)


        [y_pred_A_1_0_,y_pred_B_1_0_] = aux_model2_1_0_.predict(initial)  
        #(18,30) # environment's prediction about t+1 action based on past actions; the observation the environment choose

        out_1_0_1 = np.zeros([batch_size,1,31],dtype= 'float32') #(18,1,31)
        out_1_0_0 = np.zeros([batch_size,1,31],dtype= 'float32') #(18,1,31)
        for i in range(batch_size):
            pred_A_class_max_1_0_ = np.where(y_pred_A_1_0_[i,:] == np.max(y_pred_A_1_0_[i,:]))[0][0]
            one_hot = tf.reshape(tf.one_hot(pred_A_class_max_1_0_, 30),[1,1,30])
            out_1_0_1[i:i+1,:,:] = np.concatenate((one_hot,[[[1]]]),2) #(18,1,31)

            #pred_B_1_0_ = np.where(y_pred_B_1_0_[i,:] == np.max(y_pred_B_1_0_[i,:]))[0][0]
            pred_B_1_0_ = np.random.choice(30, 1, p=y_pred_B_1_0_[i,:])[0]
            
            one_hot = tf.reshape(tf.one_hot(pred_B_1_0_, 30),[1,1,30])
            out_1_0_0[i:i+1,:,:] = np.concatenate((one_hot,[[[0]]]),2) #(18,1,31)

        target_0_1_0 = np.reshape(out_0_1_0,[batch_size,31])[:,:-1] #(18,30) pred observation
        target_0_1_1 = np.reshape(out_0_1_1,[batch_size,31])[:,:-1] #(18,30) real action    
        target_1_0_1 = np.reshape(out_1_0_1,[batch_size,31])[:,:-1] #(18,30) pred action
        target_1_0_0 = np.reshape(out_1_0_0,[batch_size,31])[:,:-1] #(18,30) real observation

        History_0_1_ = aux_model2_0_1_.fit(initial,[target_1_0_0,target_1_0_1],verbose = 0)
        loss_0_1_ = loss_0_1_ + History_0_1_.history.get('loss')[0]

        
        if counter%50 == 0:
            print('model_0_1 - loss: ', loss_0_1_/50)
            loss_0_1_ = 0
            

        initial = tf.concat((initial[:,2:,:],out_1_0_0,out_0_1_1),1)


model_0_1 - loss:  4.762293949127197
model_0_1 - loss:  4.629613075256348
model_0_1 - loss:  4.533124465942382
model_0_1 - loss:  4.517380790710449
model_0_1 - loss:  4.430917801856995
model_0_1 - loss:  4.466203851699829
model_0_1 - loss:  4.466960411071778
model_0_1 - loss:  4.378960342407226
model_0_1 - loss:  4.3716896581649785
model_0_1 - loss:  4.386353621482849
model_0_1 - loss:  4.414221134185791
model_0_1 - loss:  4.399733715057373
model_0_1 - loss:  4.352751111984253
model_0_1 - loss:  4.371352553367615
model_0_1 - loss:  4.34528694152832
model_0_1 - loss:  4.274507508277893
model_0_1 - loss:  4.276139931678772
model_0_1 - loss:  4.338771514892578
model_0_1 - loss:  4.266030378341675
model_0_1 - loss:  4.29460120677948
model_0_1 - loss:  4.203191223144532
model_0_1 - loss:  4.285469083786011
model_0_1 - loss:  4.253345556259156
model_0_1 - loss:  4.260322179794311
model_0_1 - loss:  4.268895974159241
model_0_1 - loss:  4.230989389419555
model_0_1 - loss:  4.277942361831665
mo

In [72]:
model_0_1_.save_weights('version9_model_0_1_step5_train1_random.h5')

In [99]:
model_0_1_.load_weights('model_0_1_weight.h5')
model_1_0_.load_weights('model_1_0_weight.h5')

In [100]:
# version 10
# synchronization
# access to hidden state distribution
# one-side adaption (environment)


#step = 2
#training_times = 3;
counter = 0;
loss_1_0_ = 0


#for n in range(1,step+1):
    #for times in range(training_times):
for window in shuffle_dataset: #(18,20,31)
    initial = window


    for s in range(5): # n step forward
        counter = counter + 1


        [output_A_0_1_, output_B2_0_1_] = aux_model2_0_1_.predict(initial) 
        #(18,30) # my prediction about observations based on past observations; the action I choose

        reshape = tf.reshape(output_A_0_1_,[batch_size,1,30])
        zero = tf.reshape(tf.zeros(batch_size),[batch_size,1,1])
        out_0_1_0 = tf.concat((reshape,zero),2)
        
        out_0_1_1 = np.zeros([batch_size,1,31],dtype= 'float32') #(18,1,31)
        for i in range(batch_size):

            #pred_B_0_1_ = np.where(output_B2_0_1_[i,:] == np.max(output_B2_0_1_[i,:]))[0][0]
            pred_B_0_1_ = np.random.choice(30, 1, p=output_B2_0_1_[i,:])[0]
            
            one_hot = tf.reshape(tf.one_hot(pred_B_0_1_, 30),[1,1,30])
            out_0_1_1[i:i+1,:,:] = np.concatenate((one_hot,[[[1]]]),2) #(18,1,31)


        [y_pred_A_1_0_,y_pred_B_1_0_] = aux_model2_1_0_.predict(initial)  
        #(18,30) # environment's prediction about t+1 action based on past actions; the observation the environment choose

        reshape = tf.reshape(y_pred_A_1_0_,[batch_size,1,30])
        one = tf.reshape(tf.ones(batch_size),[batch_size,1,1])
        out_1_0_1 = tf.concat((reshape,one),2)
        
        out_1_0_0 = np.zeros([batch_size,1,31],dtype= 'float32') #(18,1,31)
        for i in range(batch_size):
            
            #pred_B_1_0_ = np.where(y_pred_B_1_0_[i,:] == np.max(y_pred_B_1_0_[i,:]))[0][0]
            pred_B_1_0_ = np.random.choice(30, 1, p=y_pred_B_1_0_[i,:])[0]
            
            one_hot = tf.reshape(tf.one_hot(pred_B_1_0_, 30),[1,1,30])
            out_1_0_0[i:i+1,:,:] = np.concatenate((one_hot,[[[0]]]),2) #(18,1,31)

        target_0_1_0 = output_A_0_1_ #(18,30) pred observation distribution
        target_0_1_1 = np.reshape(out_0_1_1,[batch_size,31])[:,:-1] #(18,30) real action    
        target_1_0_1 = y_pred_A_1_0_ #(18,30) pred action distribution
        target_1_0_0 = np.reshape(out_1_0_0,[batch_size,31])[:,:-1] #(18,30) real observation


        History_1_0_ = aux_model2_1_0_.fit(initial,[target_0_1_1,target_0_1_0],verbose = 0)
        loss_1_0_ = loss_1_0_ + History_1_0_.history.get('loss')[0]

        if counter%50 == 0:
            print('model_1_0 - loss: ', loss_1_0_/50)
            loss_1_0_ = 0

        initial = tf.concat((initial[:,2:,:],out_1_0_0,out_0_1_1),1)


model_1_0 - loss:  2.781333556175232
model_1_0 - loss:  2.7792327070236205
model_1_0 - loss:  2.747421350479126
model_1_0 - loss:  2.760349750518799
model_1_0 - loss:  2.758830370903015
model_1_0 - loss:  2.7171253681182863
model_1_0 - loss:  2.7500158739089966
model_1_0 - loss:  2.716927604675293
model_1_0 - loss:  2.7534390354156493
model_1_0 - loss:  2.747263078689575
model_1_0 - loss:  2.753672618865967
model_1_0 - loss:  2.7430478811264036
model_1_0 - loss:  2.7784336376190186
model_1_0 - loss:  2.712361888885498
model_1_0 - loss:  2.7455506181716918
model_1_0 - loss:  2.7394968128204344
model_1_0 - loss:  2.7413740444183348
model_1_0 - loss:  2.7301870489120486
model_1_0 - loss:  2.7284106016159058
model_1_0 - loss:  2.7394412088394167
model_1_0 - loss:  2.6799216079711914
model_1_0 - loss:  2.743653106689453
model_1_0 - loss:  2.719955816268921
model_1_0 - loss:  2.6828567504882814
model_1_0 - loss:  2.7159220933914185
model_1_0 - loss:  2.7091490411758423
model_1_0 - loss:  2.7

In [101]:
model_1_0_.save_weights('version10_model_1_0_step5_train1_random.h5')